In [1]:
import  numpy as np
import pandas as pd
import importlib
import time
import simuls, utils 
from pycaret.regression import *
from tqdm.notebook import tqdm
importlib.reload(simuls)
importlib.reload(utils)
#ExampleThree = simuls.ExampleThree
from simuls import *
from utils import *

In [2]:
data = ExampleThree(n = 60, sig = 1, with_test = True, seed = 1).data
data.to_df()
data.train.to_df(); data.test.to_df()
cols = [col for col in data.df.columns if "x" in col]

In [3]:
fast_models = ["lr", "br", "llar", "svm",  "en", "gbr", "omp", "kr"] # lar
slow_models = ["et", "rf", "xgboost", "mlp", "tr", ] 

In [4]:
res_df = pd.DataFrame({"name":fast_models}).set_index("name")
res_df[["model", "mse", "corr", "time"]] = None
res_df = pd.read_csv("Data/Ex3_1000_5.csv").set_index("name")

In [6]:
tracker = []
for model in ["xgboost"]:
    scores = {"mse":[], "corr":[]}
    tracker += [model]
    print("Evaluating "+ model)
    start = time.time(); n_iter = 100
    for i in tqdm(range(n_iter)):
        data = ExampleThree(n = 2000, sig=5, with_test = True, seed = i + 1).data
        data.train.to_df(); data.test.to_df()
        yt = data.test.df["y"]
        out = setup(data = data.train.df, test_data= None, target = "y", silent = True, remove_multicollinearity= True, 
            verbose = False, preprocess = False)
        tuned = tune_model(create_model(model, cross_validation = False, verbose = False), optimize = "MSE", 
            choose_better = False, verbose = False)
        preds = tuned.predict(data.test.df[cols]) 
        scores["mse"] += [mse(preds, yt)]
        scores["corr"] += [np.corrcoef(preds, yt)[0,1]]
    end = time.time() - start 
    name = get_name(tuned)
    res_df.loc[model, ["model", "mse", "corr", "time"]] = [name, np.array(scores["mse"]).mean(), np.array(scores["corr"]).mean(), end / n_iter]
    print(res_df.loc[tracker,:])
    res_df.to_csv("Data/Ex3_1000_5.csv")

    

Evaluating xgboost


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import os

In [10]:
sdf = res_df.copy()

In [29]:
#paths = [p for p in os.listdir("Data") if p.endswith("csv")]
paths = ['Ex3_200_1.csv','Ex3_1000_5.csv',]
#paths = ['Ex1_100_5.csv','Ex2_30_5.csv',]

In [30]:
df = pd.read_csv("Data/Ex3_100_1.csv").set_index("model")
for path in paths:
    if path == "Ex1_30_1.csv": continue
    x = pd.read_csv("Data/"+path)
    nam = path.split(".")[0]
    x = x.set_index("model")
    x.rename({"mse":"mse_"+nam, "corr":"corr_"+nam}, inplace=True, axis = 1)
    df = df.merge(x[["mse_"+nam, "corr_"+nam]], left_index = True, right_index = True, how="left")

In [31]:
#df.reset_index().to_csv("Data/all.csv")
#df = df.drop("name,time,corr,mse".split(","), axis =1)
df

,name,mse,corr,time,mse_Ex3_200_1,corr_Ex3_200_1,mse_Ex3_1000_5,corr_Ex3_1000_5
model,,,,,,,,
BayesianRidge,br,30.017905,0.876596,1.028677,3.380152,0.986505,28.788377,0.899897
ElasticNet,en,25.417482,0.952001,0.941001,16.622423,0.976913,16.622423,0.976913
ElasticNet,en,25.417482,0.952001,0.941001,16.622423,0.976913,37.929341,0.902952
GradientBoostingRegressor,gbr,67.545529,0.703399,3.736487,34.223526,0.875849,34.223526,0.875849
GradientBoostingRegressor,gbr,67.545529,0.703399,3.736487,34.223526,0.875849,38.442801,0.869885
KernelRidge,kr,29.231541,0.877921,1.040136,3.430934,0.986273,3.430934,0.986273
KernelRidge,kr,29.231541,0.877921,1.040136,3.430934,0.986273,29.088253,0.899133
LassoLars,llar,9.260001,0.987146,0.888546,7.962033,0.989331,31.401156,0.908435
LinearRegression,lr,28.311419,0.883181,0.899737,3.699400,0.985462,29.122335,0.899128


In [22]:
df = round(df, 4)

In [24]:
df = df.drop(["time", "name"], axis =1).reset_index()

In [25]:
to_latex(df)

LinearRegression & 16.6867 & 0.7561 & 28.6945 & 0.6367 & 33.0069 & 0.7576 \\ 
BayesianRidge & 13.0491 & 0.7839 & 27.805 & 0.6467 & 36.5027 & 0.737 \\ 
LassoLars & 14.5339 & 0.7777 & 28.5074 & 0.6514 & 44.4894 & nan \\ 
SVR & 21.0633 & 0.6558 & 33.4886 & 0.5559 & 56.8619 & 0.5395 \\ 
ElasticNet & 14.2964 & 0.7973 & 28.556 & 0.6545 & 42.7291 & nan \\ 
GradientBoostingRegressor & 20.9413 & 0.6386 & 34.1588 & 0.5656 & 61.5879 & nan \\ 
OrthogonalMatchingPursuit & 14.7276 & 0.7584 & 28.4993 & 0.6394 & 36.5252 & 0.7263 \\ 
KernelRidge & 12.8278 & 0.782 & 28.0454 & 0.6424 & 32.4948 & 0.7579


In [18]:
df

,name,mse,corr,time,mse_Ex1_100_5,corr_Ex1_100_5,mse_Ex2_30_5,corr_Ex2_30_5
model,,,,,,,,
LinearRegression,lr,16.6867,0.7561,0.6825,28.6945,0.6367,33.0069,0.7576
BayesianRidge,br,13.0491,0.7839,0.7578,27.8050,0.6467,36.5027,0.7370
LassoLars,llar,14.5339,0.7777,0.7631,28.5074,0.6514,44.4894,NaN
SVR,svm,21.0633,0.6558,1.4894,33.4886,0.5559,56.8619,0.5395
ElasticNet,en,14.2964,0.7973,1.4896,28.5560,0.6545,42.7291,NaN
GradientBoostingRegressor,gbr,20.9413,0.6386,3.2267,34.1588,0.5656,61.5879,NaN
OrthogonalMatchingPursuit,omp,14.7276,0.7584,1.4631,28.4993,0.6394,36.5252,0.7263
KernelRidge,kr,12.8278,0.7820,1.5838,28.0454,0.6424,32.4948,0.7579


In [ ]:
diabetes = datasets.load_diabetes()

In [ ]:
diabetes["data"].shape

In [ ]:
diabetes.keys()

In [ ]:
print(diabetes["DESCR"])